# 独自のデータを使用して Azure OpenAI モデルとチャットする (Add your Data)

[Azure OpenAI サービスで独自のデータを使用する - Azure OpenAI | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/cognitive-services/openai/use-your-data-quickstart?tabs=command-line&pivots=rest-api)

`Add your data` は `2023/07/13` 現在、`Azure OpenAI Serivce` 固有のAPIになります。そのため、`Python` の `OpenAI SDK` では対応していないため、`REST API` として呼び出す方法を記載しました。

## 前提条件
- `Azure` のサブスクリプションがあること
- `Azure OpenAI Serivce` が設定されていること
- `Azure Cognitive Search` インデックスを作成していること

In [28]:
import os
import requests
import json

## 必要な情報を設定します

`Azure OpenAI Service` と `Azure Cognitive Service` の設定を行います。

| 変数名 | 値 |
| ------ | -- |
| AOAIEndpoint |この値は、Azure portal から Azure OpenAI リソースを確認する際に、[Keys & Endpoint] (キーとエンドポイント) セクションで確認することができます。 または、Azure AI Studio>[チャット プレイグラウンド]>[コード ビュー] で値を確認することもできます。 エンドポイントの例: https://my-resoruce.openai.azure.com。|
| AOAIKey |この値は、Azure portal から Azure OpenAI リソースを確認する際に、[Keys & Endpoint] (キーとエンドポイント) セクションで確認することができます。 KEY1 または KEY2 を使用できます。 常に 2 つのキーを用意しておくと、サービスを中断させることなく、キーのローテーションと再生成を安全に行うことができます。|
| AOAIDeploymentId |この値は、モデルのデプロイ時にデプロイに対して選択したカスタム名に対応します。 この値は、Azure portal の [リソース管理]>[デプロイ] または Azure AI Studio の [管理]>[デプロイ] で確認できます。|
| ChatGptUrl |要求を満たすために使用される Azure OpenAI ChatGPT エンドポイント。 これは AoAIEndpoint と同じエンドポイントにすることができます。|
| ChatGptKey |ChatGptUrl と AOAIEndpoint の両方に同じ Azure OpenAI リソースを使用している場合は、AOAIKey と同じ値を使用します。|
| SearchEndpoint |この値は、Azure portal から Azure Cognitive Search リソースを確認する際に、[Keys & Endpoint] (キーとエンドポイント) セクションで確認することができます。|
| SearchKey |この値は、Azure portal から Azure Cognitive Search リソースを確認する際に、[Keys & Endpoint] (キーとエンドポイント) セクションで確認することができます。 KEY1 または KEY2 を使用できます。 常に 2 つのキーを用意しておくと、サービスを中断させることなく、キーのローテーションと再生成を安全に行うことができます。|
| SearchIndex |この値は、データを格納するために作成したインデックスの名前に対応します。 Azure portal から Azure Cognitive Search リソースを調べる場合は、[概要] セクションで確認できます。|

In [29]:
AOAIKey = input("Enter your API key: ")
AOAIEndpoint = input("Enter your endpoint: ")
AOAIDeploymentId = input("Enter your deployment name: ")
ChatGptUrl = AOAIEndpoint
ChatGptKey = AOAIKey
SearchEndpoint = input("Enter your Cognitive Search endpoint: ")
SearchKey = input("Enter your Cognitive Search API key: ")
SearchIndex = input("Enter your Cognitive Search index: ")

## URLを組み立てる

現在サポートされているバージョンは `2023-06-01-preview` のみとなります。

In [30]:
api_key = AOAIKey
api_base = AOAIEndpoint
api_version = '2023-06-01-preview'
deployment_name = AOAIDeploymentId

url = f"{api_base}openai/deployments/{deployment_name}/extensions/chat/completions?api-version={api_version}"

#print(url)

## ヘッダを組み立てる

In [31]:
chatgpt_url = ChatGptUrl
chatgpt_key = ChatGptKey

headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
    "chatgpt_url": chatgpt_url,
    "chatgpt_key": chatgpt_key,
}

#print(json.dumps(headers, indent=4))

## パラメータを組み立てます

### パラメータ

| パラメーター | Type | 必須 | Default | 説明 | 
| ------------ | ---- | ---- | ------- | ---- |
| messages | array | 必須 | null | チャット形式でチャット入力候補を生成するメッセージ。 | 
| dataSources | array | 必須 |  | データ特徴量で Azure OpenAI に使用するデータ ソース。 | 
| temperature | number | オプション | 0 | 使用するサンプリング温度 (0 から 2)。 0.8 のような大きい値にすると、出力はよりランダムになり、0.2 のような小さい値にすると、出力はより集中的および決定論的になります。 一般に、これと top_p の両方ではなく、いずれかを変更することをお勧めします。 | 
| top_p | number | オプション | 1 | 核サンプリングと呼ばれる、温度によるサンプリングの代替の場合、モデルでは top_p 確率質量を持つトークンの結果が考慮されます。 したがって、0.1 は、上位 10% の確率質量を含むトークンのみが考慮されることを意味します。 一般的に、これと temperature の両方ではなく、いずれかを変更することをお勧めします。 | 
| stream | boolean | 省略可能 | false | 設定すると、ChatGPT と同様に部分的なメッセージ デルタが送信されます。 トークンは、使用可能になると data-only server-sent イベントとして送信され、ストリームは "messages": [{"delta": {"content": "[DONE]"}, "index": 2, "end_turn": true}] メッセージで終了します。 | 
| stop | 文字列または配列 | オプション | null | API がそれ以上のトークンの生成を停止する、最大 2 つのシーケンス。 | 
| max_tokens | 整数 (integer) | 省略可能 | 1000 | 生成された回答に許可されるトークンの最大数。 既定では、モデルが返すことができるトークンの数は 4096 - prompt_tokens になります。|

### dataSources 内の parameters フィールド

| パラメーター | Type | 必須 | Default | 説明 |
| ------------ | ---- | ---- | ------- | ---- | 
| type | string | 必須 | null | データ特徴量で Azure OpenAI に使用するデータ ソース。 Azure Cognitive Search の場合、値は AzureCognitiveSearch です。 | 
| endpoint | string | 必須 | null | データ ソース エンドポイント。 | 
| key | string | 必須 | null | サービスの Azure Cognitive Search 管理者キーの 1 つ。 | 
| indexName | string | 必須 | null | 使用する検索インデックス。 | 
| fieldsMapping | ディクショナリ | オプション | null | インデックス データ列マッピング。 | 
| inScope | boolean | 省略可能 | true | 設定した場合、この値によって、根拠付けるデータ コンテンツに固有の応答が制限されます。 | 
| topNDocuments | number | オプション | 5 | ドキュメント拡張のためにフェッチする必要があるドキュメントの数。 | 
| queryType | string | 省略可能 | simple | Azure Cognitive Search に使用するクエリ オプションを示します。 | 
| semanticConfiguration | string | 省略可能 | null | セマンティック検索の構成。 queryType が semantic に設定されている場合にのみ使用できます。 | 
| roleInformation | string | 省略可能 | null | どう振る舞うべきかに関する指示と、応答の生成時に参照する必要があるコンテキストをモデルに渡します。 Azure OpenAI Studio の "システム メッセージ" に相当します。 トークン制限 100 が存在し、全体的なトークン制限に反映されます。 | 



In [32]:
cognitive_search_endpoint = SearchEndpoint
cognitive_search_api_key = SearchKey
cognitive_search_index = SearchIndex

data = {
    "messages": [
        {
            "role": "user",
            "content": "What are the differences between Azure Machine Learning and Azure Cognitive Services?"
        }
    ],
    "dataSources": [
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": cognitive_search_endpoint,
                "key": cognitive_search_api_key,
                "indexName": cognitive_search_index
            }
        }
    ]
}

#print(json.dumps(data, indent=4))

## 結果を取得します

In [33]:
response = requests.post(url, headers=headers, json=data)

print(json.dumps(response.json(),indent=4))

{
    "id": "ceeb9619-0cb5-48fb-9671-1224077b51dd",
    "model": "gpt-35-turbo",
    "created": 1689254837,
    "object": "chat.completion",
    "choices": [
        {
            "index": 0,
            "messages": [
                {
                    "index": 0,
                    "role": "tool",
                    "content": "{\"citations\": [], \"intent\": \"[\\\"What are the differences between Azure Machine Learning and Azure Cognitive Services?\\\"]\"}",
                    "end_turn": false
                },
                {
                    "index": 1,
                    "role": "assistant",
                    "content": "Azure Machine Learning is a cloud-based service that enables you to build, train, and deploy machine learning models. Azure Cognitive Services, on the other hand, provides pre-built APIs that enable you to add intelligent features to your applications without requiring any machine learning expertise. While Azure Machine Learning is focused on buil